<a href="https://colab.research.google.com/github/Darkdew/CNNHPTuningusingKERAS_Final/blob/master/HPTuningKerasCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=1ec5415f50ab5a9e83a805af65f6970be95d18ab884c5903ff9e185f35d36285
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ea67f69a098eee5a0a7bb0791ae20b9106b4f004574498396d986385eb41f8aa
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0


In [0]:
#Building a CNN Model-----------------------------------------------------

fashion_mnist= keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels),(test_images, test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
#lets scale our images to bring their value from [0,255] to [0,1]
#this will make our calculations simpler
train_images= train_images/255.0
test_images= test_images/255.0

In [7]:
#lets check how our first iamge looks like
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
#and the shape of the first image
train_images[0].shape

(28, 28)

In [0]:
#in the input layer of CNN we need to provide- count of images, pixels, the 4 dimensions
# So lets reshape our arrays in the same manner
train_images= train_images.reshape(len(train_images), 28, 28, 1)
test_images= test_images.reshape(len(test_images), 28, 28, 1)

In [10]:
train_images.shape

(60000, 28, 28, 1)

In [11]:
test_images.shape

(10000, 28, 28, 1)

In [0]:
def build_model(hp):
  model= keras.Sequential([
                           keras.layers.Conv2D(
                               filters= hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                               kernel_size= hp.Choice('conv_1_kernel', values=[3,5]),
                                activation='relu', 
                                kernel_initializer='glorot_uniform',
                                input_shape= (28,28,1)

                           ),

                          keras.layers.Conv2D(
                              filters= hp.Int('conv_2_filter', min_value=32, max_value= 128, step=16),
                              kernel_size= hp.Choice('conv_2_kernel', values=[3,5]),
                              activation='relu'
                          ) ,
                           
                           
                           keras.layers.Flatten(),
                          
                           keras.layers.Dense(
                               units= hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                               activation='relu'
                           ),
                           keras.layers.Dense(
                               10, activation='softmax'
                           )

                               
                           
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values= [1e-2,1e-3])),
                loss= 'sparse_categorical_crossentropy',
                metrics= ['accuracy'])
  return model

In [13]:
!pip install keras-tuner

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
#Lets work on the Random Search now
tuner_search=RandomSearch(build_model, 
                          objective="val_accuracy", 
                          max_trials=5, 
                          directory='/content/drive/My Drive/Colab Notebooks/CNNHPTuning', 
                          project_name='MNIST_fashion' )

In [16]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3926 - accuracy: 0.8598 - val_loss: 0.3194 - val_accuracy: 0.8807
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2548 - accuracy: 0.9068 - val_loss: 0.2668 - val_accuracy: 0.9013
Epoch 3/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1980 - accuracy: 0.9270 - val_loss: 0.2474 - val_accuracy: 0.9103


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5339 - accuracy: 0.8104 - val_loss: 0.4210 - val_accuracy: 0.8468
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4072 - accuracy: 0.8487 - val_loss: 0.3895 - val_accuracy: 0.8552
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3805 - accuracy: 0.8601 - val_loss: 0.4296 - val_accuracy: 0.8417


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.4816 - accuracy: 0.8341 - val_loss: 0.3582 - val_accuracy: 0.8700
Epoch 2/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3505 - accuracy: 0.8715 - val_loss: 0.3422 - val_accuracy: 0.8697
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3195 - accuracy: 0.8821 - val_loss: 0.3338 - val_accuracy: 0.8790


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.6315 - accuracy: 0.8126 - val_loss: 0.4407 - val_accuracy: 0.8287
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3860 - accuracy: 0.8565 - val_loss: 0.3754 - val_accuracy: 0.8612
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.3602 - accuracy: 0.8665 - val_loss: 0.4337 - val_accuracy: 0.8540


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3931 - accuracy: 0.8603 - val_loss: 0.2941 - val_accuracy: 0.8883
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2477 - accuracy: 0.9096 - val_loss: 0.2641 - val_accuracy: 0.9005
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1923 - accuracy: 0.9288 - val_loss: 0.2515 - val_accuracy: 0.9098


INFO:tensorflow:Oracle triggered exit


In [0]:
model= tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,565,050
Trainable params: 1,565,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Lets train our mdel now using the best hyperparameters achieved
model.fit(train_images, train_labels, epochs=10,validation_split= 0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1494 - accuracy: 0.9448 - val_loss: 0.2733 - val_accuracy: 0.9058
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1128 - accuracy: 0.9578 - val_loss: 0.2774 - val_accuracy: 0.9150
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0822 - accuracy: 0.9694 - val_loss: 0.3282 - val_accuracy: 0.9083
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0599 - accuracy: 0.9775 - val_loss: 0.3444 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 0.3559 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0350 - accuracy: 0.9880 - val_loss: 0.4562 - val_accuracy: 0.9098
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0297 - accuracy: 0.9899 - val_loss: 0.5149 - val_accuracy